# Set constants

In [2]:
test_data_path = "./tmp/test.csv"

# Load test data

In [3]:
import yfinance as yf
import pandas as pd

In [4]:
if 1 == 0:
    data_df:pd.DataFrame =  yf.Ticker("EURUSD=X").history(start="2023-01-01",end="2024-01-01", interval="1d")
    data_df.index = data_df.index.date
    data_df.index.name = "Date"
    data_df["High"] = data_df[["Open","High","Low","Close"]].max(axis=1)
    data_df["Low"] = data_df[["Open","High","Low","Close"]].min(axis=1)
    data_df["Volume"] = 10000000
    data_df.to_csv(test_data_path)

In [5]:
data_df:pd.DataFrame = pd.read_csv(test_data_path)
data_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-01-02,1.070973,1.071237,1.065326,1.070973,10000000,0.0,0.0
1,2023-01-03,1.067771,1.068262,1.052155,1.067771,10000000,0.0,0.0
2,2023-01-04,1.054685,1.063151,1.054596,1.054685,10000000,0.0,0.0
3,2023-01-05,1.060637,1.063264,1.051558,1.060637,10000000,0.0,0.0
4,2023-01-06,1.052222,1.062225,1.048526,1.052222,10000000,0.0,0.0
...,...,...,...,...,...,...,...,...
255,2023-12-25,1.102657,1.104240,1.099989,1.102657,10000000,0.0,0.0
256,2023-12-26,1.102026,1.103997,1.100958,1.102026,10000000,0.0,0.0
257,2023-12-27,1.104301,1.112248,1.102925,1.104301,10000000,0.0,0.0
258,2023-12-28,1.110864,1.113945,1.107101,1.110864,10000000,0.0,0.0


# Set strategy

In [6]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma
from pyalgotrade.broker.backtesting import Broker
from pyalgotrade.bar import BasicBar
from pyalgotrade.strategy.position import LongPosition, Position, ShortPosition
from pyalgotrade.broker import OrderExecutionInfo, Order

In [11]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        # We want a 15 period SMA over the closing prices.
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), 15)
        self.__instrument = instrument
        self.__position:Position = None

    def _action_to_stirng(self, action):
        if action == Order.Action.BUY:
            return "BUY"
        elif action == Order.Action.SELL:
            return "SELL"
        elif action == Order.Action.SELL_SHORT:
            return "SELL_SHORT"
        elif action == Order.Action.BUY_TO_COVER:
            return "BUY_TO_COVER"
        else:
            raise Exception(f"Unexpected action {action}")

    def onEnterOk(self, position):
        order: Order = position.getEntryOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()

        self.info(f"{self._action_to_stirng(order.getAction())} at {execInfo.getPrice()}")

    def onEnterCanceled(self, position:Position):
        self.__position = None

    def onExitOk(self, position:Position):
        order: Order = position.getExitOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()
        self.info(f"Close {self._action_to_stirng(order.getAction())} order at {execInfo.getPrice()}")
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        broker:Broker = self.getBroker()
        equity = broker.getEquity()

        bar: BasicBar = bars[self.__instrument]
        
        if self.__sma[-1] is None:
            return
        
        
        if bar.getPrice() > self.__sma[-1]:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 100000, True)
            elif isinstance(self.__position, ShortPosition) and not self.__position.exitActive():
                self.__position.exitMarket()
        elif bar.getPrice() < self.__sma[-1]:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterShort(self.__instrument, 100000, True)
            elif isinstance(self.__position, LongPosition) and not self.__position.exitActive():
                self.__position.exitMarket()


In [12]:

# Load the bar feed from the CSV file
feed = quandlfeed.Feed()
feed.addBarsFromCSV("orcl", test_data_path)

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "orcl")
myStrategy.run()

2023-01-23 00:00:00 strategy [INFO] BUY at 1.0864723920822144
2023-02-07 00:00:00 strategy [INFO] Close SELL order at 1.0730764865875244
2023-02-08 00:00:00 strategy [INFO] SELL_SHORT at 1.0729728937149048
2023-03-03 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.0599732398986816
2023-03-06 00:00:00 strategy [INFO] SELL_SHORT at 1.0626201629638672
2023-03-08 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.05507493019104
2023-03-09 00:00:00 strategy [INFO] SELL_SHORT at 1.0549525022506714
2023-03-14 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.0725010633468628
2023-03-15 00:00:00 strategy [INFO] BUY at 1.0727657079696655
2023-03-17 00:00:00 strategy [INFO] Close SELL order at 1.0614134073257446
2023-03-20 00:00:00 strategy [INFO] SELL_SHORT at 1.0678855180740356
2023-03-21 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.071914792060852
2023-03-22 00:00:00 strategy [INFO] BUY at 1.0774236917495728
2023-05-03 00:00:00 strategy [INFO] Close SELL order at 1.1

In [13]:
myStrategy.getBroker().getShares("orcl")

100000

In [14]:
myStrategy.getBroker().getEquity()

1006301.6891479492

In [29]:
data_df[data_df["High"] < data_df["Low"]]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits


# Define trading simulator

In [ ]:
from src.simulation.abs_trading_simulatior import absTradingSimulatior,StrategyId,SimulationConfig,SimulationLog
from src.simulation.config

In [ ]:
class TradingSimulatior(absTradingSimulatior):
    def strategy_id(self)->StrategyId:
        return StrategyId("Example Strategy", "v1")
    
    def _run(self, run_config: SimulationConfig)->SimulationLog:
        # Load the bar feed from the CSV file
        feed = quandlfeed.Feed()
        feed.addBarsFromCSV("EURUSD", test_data_path)

        # Evaluate the strategy with the feed's bars.
        myStrategy = MyStrategy(feed, "EURUSD")
        myStrategy.run()

# Optimizing

In [ ]:
from src.optimization_analyzer.optimization_analyzer import OptimizationAnalyzer, OptimizationConfig

In [ ]:
simulation_report_factory: absTradingSimulatior

opt_config = OptimizationConfig()

In [ ]:
opt_analyzer = OptimizationAnalyzer()
opt_analyzer.analis_optimization_flow()